### importando librerias

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from imblearn.under_sampling import RandomUnderSampler
from torch.utils.data import WeightedRandomSampler

### definiendo directorios

In [ ]:
# Directorios de las imágenes
train_violent_dir = "../resources/dataset/train/violencia/"
train_non_violent_dir = "../resources/dataset/train/no_violencia/"
test_violent_dir = "../resources/dataset/test/violencia/"
test_non_violent_dir = "../resources/dataset/test/no_violencia/"

### Clase personalizada para cargar el dataset

In [ ]:

class CustomImageDataset(Dataset):
    def __init__(self, violent_dir, non_violent_dir, transform=None):
        self.img_labels = []
        self.img_paths = []
        for filename in os.listdir(violent_dir):
            if filename.endswith(".jpg"):
                self.img_labels.append(1)  # Violenta
                self.img_paths.append(os.path.join(violent_dir, filename))
        for filename in os.listdir(non_violent_dir):
            if filename.endswith(".jpg"):
                self.img_labels.append(0)  # No violenta
                self.img_paths.append(os.path.join(non_violent_dir, filename))
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image = Image.open(self.img_paths[idx]).convert('RGB')
        label = self.img_labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

### Transformaciones para las imágenes

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalización para VGG16
])

### Crear los datasets

In [ ]:

train_dataset = CustomImageDataset(train_violent_dir, train_non_violent_dir, data_transforms)
test_dataset = CustomImageDataset(test_violent_dir, test_non_violent_dir, data_transforms)

### Aplicar ROS 

In [ ]:

X_resampled, y_resampled = RandomUnderSampler(random_state=42).fit_resample(
    np.arange(len(train_dataset)).reshape(-1, 1), train_dataset.img_labels
)
X_resampled = X_resampled.flatten()
train_sampler = WeightedRandomSampler(
    weights=[1 if i in X_resampled else 0 for i in range(len(train_dataset))],
    num_samples=len(train_dataset),
    replacement=True
)

### Crea dataloaders

In [ ]:
#train_loader = DataLoader(train_dataset, batch_size=64, sampler=None, drop_last=True,shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=True)
train_loader = DataLoader(train_dataset, batch_size=192, sampler=train_sampler, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=192, shuffle=False, drop_last=True)

## Crear el modelo

### Cargar el modelo VGG16 pre-entrenado con weights

In [ ]:
# Cargar el modelo VGG16 pre-entrenado con weights                    MAS RECIENTE
model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)

In [ ]:
#model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
#num_ftrs = model.classifier[6].in_features
#model.classifier[6] = nn.Linear(num_ftrs, 2)

### Congelar capas (opcional)

In [ ]:
for param in model.features.parameters():
    param.requires_grad = False

### Definir el dispositivo (GPU si está disponible)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
if torch.cuda.is_available():
    print("CUDA está disponible. PyTorch puede utilizar la GPU.")
    print(f"Número de dispositivos CUDA disponibles: {torch.cuda.device_count()}")
    print(f"Dispositivo CUDA actual: {torch.cuda.current_device()}")
    print(f"Nombre del dispositivo CUDA actual: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("CUDA no está disponible. PyTorch utilizará la CPU.")

### Definir criterio de pérdida y optimizador

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Entrenamiento

In [ ]:

train_losses, train_accs = [], []
val_losses, val_accs = [], []
for epoch in range(4):  # 4 épocas de entrenamiento
    model.train()  # Poner el modelo en modo entrenamiento
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:  # Iterar sobre el dataloader de entrenamiento
        inputs = inputs.to(device)  # Mover los datos de entrada al dispositivo (GPU/CPU)
        labels = labels.to(device)

        optimizer.zero_grad()  # Poner a cero los gradientes
        outputs = model(inputs)  # Obtener predicciones
        loss = criterion(outputs, labels)  # Calcular la pérdida
        loss.backward()  # Retropropagación
        optimizer.step()  # Actualizar los parámetros del modelo

        running_loss += loss.item()  # Acumular la pérdida
        _, predicted = torch.max(outputs, 1)  # Obtener las clases predichas
        total += labels.size(0)  # Contar el número total de muestras
        correct += (predicted == labels).sum().item()  # Contar las predicciones correctas

    train_loss = running_loss / len(train_loader)  # Calcular la pérdida promedio de entrenamiento
    train_acc = 100 * correct / total  # Calcular la precisión de entrenamiento
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Validación (en cada época)
    model.eval()  # Poner el modelo en modo evaluación
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():  # No calcular gradientes durante la validación
        for inputs, labels in test_loader:  # Iterar sobre el dataloader de validación
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()  # Acumular la pérdida de validación
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss /= len(test_loader)  # Calcular la pérdida promedio de validación
    val_acc = 100 * val_correct / val_total  # Calcular la precisión de validación
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f'Epoch [{epoch+1}/{4}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')



### Guardar el modelo

In [ ]:
# Guardar el modelo
torch.save(model.state_dict(), "../modelos/VGG16_pytorch.pth")

# Visualización de la pérdida y precisión
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.legend()
plt.title("Loss")

plt.subplot(1, 2, 2)
plt.plot(train_accs, label="Train Accuracy")
plt.plot(val_accs, label="Validation Accuracy")
plt.legend()
plt.title("Accuracy")

plt.show()


### Evaluación en el conjunto de prueba

In [ ]:

model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Matriz de confusión
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=["No Violencia", "Violencia"], yticklabels=["No Violencia", "Violencia"])
plt.xlabel("Clase Predicha")
plt.ylabel("Clase Real")
plt.title("Matriz de Confusión")
plt.show()

# Accuracy en el conjunto de prueba
test_accuracy = 100 * np.mean(np.array(y_true) == np.array(y_pred))
print(f"Test Accuracy: {test_accuracy:.2f}%")


###  Metricas para conjuntos de datos desbalanceados

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
# Métricas para conjuntos de datos desbalanceados
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Métricas para conjuntos de datos desbalanceados
conf_matrix = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
gmean = np.sqrt(recall * (tn / (tn + fp)))  # G-Mean

print(f"Test Accuracy: {100 * np.mean(np.array(y_true) == np.array(y_pred)):.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
